In [1]:
import elotl.nahuatl as nahuatl

load an english to spanish transformer

In [8]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

nllb_tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
nllb = AutoModelForSeq2SeqLM.from_pretrained(
    "facebook/nllb-200-distilled-600M")

eng_text = "coyote plant"
inputs = nllb_tokenizer(f"eng_Latn {eng_text}", return_tensors="pt")
translated = nllb.generate(
    **inputs,
    forced_bos_token_id=nllb_tokenizer.convert_tokens_to_ids("spa_Latn"),
    max_length=30
)
spanish_text = nllb_tokenizer.batch_decode(
    translated, skip_special_tokens=True)[0]
print(spanish_text)

planta de coyote


now load spanish to nahuatl model

In [6]:
# Load model directly
from transformers import  AutoModelForSeq2SeqLM

t5_tokenizer = AutoTokenizer.from_pretrained(
    "somosnlp-hackathon-2022/t5-small-spanish-nahuatl")
t5_model = AutoModelForSeq2SeqLM.from_pretrained(
    "somosnlp-hackathon-2022/t5-small-spanish-nahuatl")

In [9]:
input_ids = t5_tokenizer('translate Spanish to Nahuatl: ' +
                          spanish_text, return_tensors='pt').input_ids
outputs = t5_model.generate(input_ids)
print(outputs)
decoded_output = t5_tokenizer.batch_decode(outputs, skip_special_tokens=True)[
    0]  # Decode the tokens to text
print(decoded_output)

tensor([[  0,   3, 157,  32,  63,  32,  17,  40,   3,  17,  40, 144,  40, 138,
         173,  23,   1]])
koyotl tlatlalili


ok not great but kind of expected

In [2]:
from load_data import load_text_files, load_subchapter_data

file_names, subchapter_names, texts = load_text_files()
subchapter_data = load_subchapter_data()

nahuatl_texts = subchapter_data["nahuatl"]
print(nahuatl_texts)

0            Aca-capac-quili-tl
1                        Aca-tl
2                     A-chil-li
3      Acocotli; Acoco-xihui-tl
4                     Acxoya-tl
                 ...           
305                    coco-tli
306                    pezo-tli
307              acuecueyalo-tl
308                      oc-tli
309                tla-qua-tzin
Name: nahuatl, Length: 310, dtype: object


In [4]:
import re
import pandas as pd

def segment_nahuatl(text):
    """Segments Nahuatl text into morphemes using common patterns"""
    suffixes = ['tzin', 'tli', 'tin', 'que', 'pan', 'can', 'tic', 'toc',
                'tia', 'lia', 'hui', 'tla', 'tli', 'tl', 'li', 'in']
    prefixes = ['ni', 'ti', 'xi', 'mo', 'no', 'to', 'ne', 'te', 'tla', 'on']

    words = str(text).split()
    segmented_words = []

    for word in words:
        if len(word) < 3:
            segmented_words.append(word)
            continue

        segmented = word
        for suffix in suffixes:
            if segmented.endswith(suffix):
                segmented = segmented[:-len(suffix)] + '-' + suffix
                break

        for prefix in prefixes:
            if segmented.startswith(prefix):
                segmented = prefix + '-' + segmented[len(prefix):]
                break

        segmented_words.append(segmented)

    return ' '.join(segmented_words)


def load_segmented_subchapter_data(filepath="./verify_subchapter.csv"):
    """Loads and processes subchapter data from CSV, returning relevant columns"""
    df = pd.read_csv(filepath)
    subchapter = df["subchapter"].str.split(";", expand=True).fillna(0)

    nahuatl = df["nahuatl"].apply(segment_nahuatl)

    return {
        'subchapter': subchapter,
        'nahuatl': nahuatl,
        'official_name': df["official_name"],
        'ID': df["ID"],
        'type': df["type"]
    }

file_names, subchapter_names, texts = load_text_files()
segmented_subchapter_data = load_segmented_subchapter_data()

regex_nahuatl_texts = segmented_subchapter_data["nahuatl"]
print(regex_nahuatl_texts)

0            Aca-capac-quili-tl
1                        Aca-tl
2                     A-chil-li
3      Acocotli; Acoco-xihui-tl
4                     Acxoya-tl
                 ...           
305                    coco-tli
306                    pezo-tli
307              acuecueyalo-tl
308                      oc-tli
309                tla-qua-tzin
Name: nahuatl, Length: 310, dtype: object


In [5]:
from gensim.models import FastText

sentences = []
for text in regex_nahuatl_texts:
    morphemes = [morpheme for word in text.split()
                 for morpheme in word.split('-')]
    sentences.append(morphemes)

model = FastText(
    sentences,
    vector_size=100,
    window=3,
    min_count=1,
    min_n=3,
    max_n=6,
    workers=4
)

model.save("nahuatl_morpheme_embeddings.bin")

test_morphemes = ["tlal", "xochi", "tl"]
for morpheme in test_morphemes:
    if morpheme in model.wv:
        print(f"\nSimilar to {morpheme}:")
        similar = model.wv.most_similar(morpheme)
        for word, score in similar[:3]:
            print(f"{word}: {score:.4f}")


Similar to tlal:
matlal: 0.3658
Matlal: 0.3502
tla: 0.3403

Similar to xochi:
xochitI: 0.5757
xochiti: 0.4645
can: 0.3305

Similar to tl:
tli: 0.3892
tlemai: 0.2621
Coz: 0.2607


In [6]:
print(model.wv.most_similar("xochi", topn=5))

[('xochitI', 0.5757061243057251), ('xochiti', 0.4644985496997833), ('can', 0.33054521679878235), ('Cólto', 0.2992047667503357), ('Eca', 0.2878001928329468)]
